# Fine tune language models

In [1]:
from argparse import Namespace
import copy
import pandas as pd
from gensim.models import Word2Vec
from gensim import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
#nlp = spacy.load("en_core_web_lg")

## Initialize a pre-trained language model

In [3]:
# model types
# w2v: word2vec
# ft: fasttext
model_type = "w2v"   
model_path = "/Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model"

if model_type.lower() in ["w2v", "word2vec"]:
    # Word2Vec
    embedding_model_orig = Word2Vec.load(model_path)
elif model_type.lower() in ["ft", "fasttext"]:
    # FastText
    embedding_model_orig = FastText.load(model_path)
print(embedding_model_orig)

2019-11-20 18:22:44,514 : INFO : loading Word2Vec object from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model
2019-11-20 18:22:45,462 : INFO : loading wv recursively from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model.wv.* with mmap=None
2019-11-20 18:22:45,463 : INFO : loading vectors from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model.wv.vectors.npy with mmap=None
2019-11-20 18:22:45,797 : INFO : setting ignored attribute vectors_norm to None
2019-11-20 18:22:45,798 : INFO : loading vocabulary recursively from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/m

Word2Vec(vocab=434049, size=300, alpha=0.03)


In [4]:
min_alpha_yet_reached = embedding_model_orig.min_alpha_yet_reached
min_alpha_yet_reached

0.0007000009816451012

## Read DB

In [5]:
db_sentence_orig = pd.read_pickle("./db_trove_v002.pkl")

# Quality bands 3, 4

In [6]:
db_sentence = db_sentence_orig[(db_sentence_orig['quality_band'] == 3) | 
                               (db_sentence_orig['quality_band'] == 4)]
db_sentence['quality_band'].value_counts()

3    3600
4    1495
Name: quality_band, dtype: int64

## preprocess DB

In [7]:
"""
def cleanup(myrow, colname="corrected"):
    # remove all # and @§
    
    corpus = [re.sub(r'#', '', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [re.sub(r'@', '', element, flags=re.IGNORECASE) for element in corpus]
    
    # --- remove 2 or more .
    corpus = [re.sub('[.]{2,}', '.', element) for element in corpus]
    # --- add a space before and after a list of punctuations
    corpus = [re.sub(r"([.,!?:;\"\'])", r" \1 ", element) for element in corpus]
    # --- remove everything except:
    #corpus = [re.sub(r"([^a-zA-Z\-.:;,!?\d+]+)", r" ", element) for element in corpus]
    corpus = [re.sub(r"([^a-zA-Z\d+]+)", r" ", element) for element in corpus]
    # --- replace numbers with <NUM>
    corpus = [re.sub(r'\b\d+\b', '<NUM>', element) for element in corpus]
    corpus = [re.sub('--', '', element) for element in corpus]
    # --- normalize white spaces
    corpus = [re.sub('\s+', ' ', element) for element in corpus]
    
    # remove multiple spaces
    corpus = [re.sub(r'\s+', ' ', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [element.strip() for element in corpus]
    #corpus = [element.lower() for element in corpus]
    return corpus
"""

'\ndef cleanup(myrow, colname="corrected"):\n    # remove all # and @§\n    \n    corpus = [re.sub(r\'#\', \'\', element, flags=re.IGNORECASE) for element in corpus]\n    corpus = [re.sub(r\'@\', \'\', element, flags=re.IGNORECASE) for element in corpus]\n    \n    # --- remove 2 or more .\n    corpus = [re.sub(\'[.]{2,}\', \'.\', element) for element in corpus]\n    # --- add a space before and after a list of punctuations\n    corpus = [re.sub(r"([.,!?:;"\'])", r" \x01 ", element) for element in corpus]\n    # --- remove everything except:\n    #corpus = [re.sub(r"([^a-zA-Z\\-.:;,!?\\d+]+)", r" ", element) for element in corpus]\n    corpus = [re.sub(r"([^a-zA-Z\\d+]+)", r" ", element) for element in corpus]\n    # --- replace numbers with <NUM>\n    corpus = [re.sub(r\'\x08\\d+\x08\', \'<NUM>\', element) for element in corpus]\n    corpus = [re.sub(\'--\', \'\', element) for element in corpus]\n    # --- normalize white spaces\n    corpus = [re.sub(\'\\s+\', \' \', element) for elem

In [8]:
def cleanup(myrow, col_name):
    all_clean_rows = []
    for sent in myrow[col_name]:
        one_clean_row = []
        for token in sent:
            one_clean_row.append(token.lower())
        all_clean_rows.append(one_clean_row)
    return all_clean_rows

In [9]:
db_sentence["ocr_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["ocr_sentencizer"], axis=1)
db_sentence["corrected_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["corrected_sentencizer"], axis=1)

/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
print(db_sentence.shape)
db_sentence.head()

(5095, 25)


,filePath,articleId,articleType,year,ocrText,humanText,corrected,str_similarity,str_length_humanText,str_length_ocrText,...,ocr_dict_lookup,ocr_dict_lookup_list,ocr_dict_perc,corrected_dict_lookup_list,corr_dict_perc,corrected_sentencizer_list,ocr_sentencizer_list,jaccard_similarity,ocr_sentencizer_cleaned,corrected_sentencizer_cleaned
3,./trove_overproof/datasets/dataset1/rawTextAnd...,18366055,Article,1953,FIRST CHURCH I SERVICE 1 Presbyterian I ' Anni...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,0.738901,946,832,...,"[[5, 6, 1, 7, 1, 12, 1, 1, 11, 1, 3, 5, -12, 3...","[5, 6, 1, 7, 1, 12, 1, 1, 11, 1, 3, 5, -12, 3,...",82.000000,"[5, 6, 7, 12, 11, 3, 5, 11, 2, 3, 5, 12, 6, 7,...",99.206349,"[FIRST, CHURCH, SERVICE, Presbyterian, Anniver...","[FIRST, CHURCH, I, SERVICE, 1, Presbyterian, I...",0.191919,"[[first, church, i, service, 1, presbyterian, ...","[[first, church, service, presbyterian, annive..."
4,./trove_overproof/datasets/dataset1/rawTextAnd...,18386137,Article,1953,"""Bob"" Lulham's Fight Against Thallium District...","""Bob"" Lulham's Fight Against Thallium Arthur ...","""Bob"" Lulham's Fight Against Thallium Arthur ...",0.493898,2950,2740,...,"[[1, 3, 1, -6, 2, 5, 7, 8, 8, 8, 1], [4, 5, 6,...","[1, 3, 1, -6, 2, 5, 7, 8, 8, 8, 1, 4, 5, 6, 6,...",95.580110,"[1, 3, 1, -6, 2, 5, 7, 8, 6, 6, 1, 1, 3, 1, 1,...",95.652174,"["", Bob, "", Lulham, 's, Fight, Against, Thalli...","["", Bob, "", Lulham, 's, Fight, Against, Thalli...",0.282682,"[["", bob, "", lulham, 's, fight, against, thall...","[["", bob, "", lulham, 's, fight, against, thall..."
10,./trove_overproof/datasets/dataset1/rawTextAnd...,18391223,Article,1953,"rchitect For lympics aid £31,665 - ?? MELBOURN...","Architect For Olympics paid £31,665 MELBOURNE...","Architect For Olympics paid £31,665 MELBOURNE...",0.793510,678,585,...,"[[-8, 3, 7, 3, 1, -6, 1, 2, 9, 1, 6], [1, 1, 5...","[-8, 3, 7, 3, 1, -6, 1, 2, 9, 1, 6, 1, 1, 5, 5...",88.607595,"[9, 3, 8, 4, 1, -6, 9, 1, 6, 2, 5, 5, 1, 3, 9,...",94.680851,"[Architect, For, Olympics, paid, £, 31,665, ME...","[rchitect, For, lympics, aid, £, 31,665, -, ??...",0.302885,"[[rchitect, for, lympics, aid, £, 31,665, -, ?...","[[architect, for, olympics, paid, £, 31,665, m..."
21,./trove_overproof/datasets/dataset1/rawTextAnd...,18392087,Article,1953,"Mint Building i Sur,--Tb* cía .\-fint Build- i...","Mint Building Sir, - The old Mint Build- ing i...","Mint Building Sir, - The old Mint Build- ing i...",0.687574,813,845,...,"[[4, 8, 1, -8, 1, 3, 1, 1, 1, 4, 5, 1, 3, 2, 9...","[4, 8, 1, -8, 1, 3, 1, 1, 1, 4, 5, 1, 3, 2, 9,...",64.347826,"[4, 8, 3, 1, 1, 3, 3, 4, 5, 1, 3, 2, 9, 6, 1, ...",98.333333,"[Mint, Building, Sir, ,, -, The, old, Mint, Bu...","[Mint, Building, i, Sur,--Tb, *, cía, ., \, -,...",0.163399,"[[mint, building, i, sur,--tb, *, cía, ., \, -...","[[mint, building, sir, ,, -, the, old, mint, b..."
25,./trove_overproof/datasets/dataset1/rawTextAnd...,27904257,Article ILLUSTRATED,1947,"PEARL TOWN PAINTED BY ELIZABETH DURACK ""The st...","PEARL TOWN PAINTED BY ELIZABETH DURACK ""The st...","PEARL TOWN PAINTED BY ELIZABETH DURACK ""The st...",0.744510,2732,2722,...,"[[5, 4, 7, 2, 9, 6, 1, 3, 5, 2, 3, 5, 4, 8, 4,...","[5, 4, 7, 2, 9, 6, 1, 3, 5, 2, 3, 5, 4, 8, 4, ...",96.800000,"[5, 4, 7, 2, 9, 6, 1, 3, 5, 2, 3, 5, 4, 8, 4, ...",99.726776,"[PEARL, TOWN, PAINTED, BY, ELIZABETH, DURACK, ...","[PEARL, TOWN, PAINTED, BY, ELIZABETH, DURACK, ...",0.236032,"[[pearl, town, painted, by, elizabeth, durack,...","[[pearl, town, painted, by, elizabeth, durack,..."


## Update a pre-trained LM

In [11]:
# args for Word2Vec
w2v_args = Namespace(
    epochs=5, 
    # only for Word2Vec
    compute_loss=True,                               # If True, computes and stores loss value which can be retrieved using get_latest_training_loss().

#     size=100,                                        # Dimensionality of the word vectors.
#     alpha=0.03,                                      # The initial learning rate.
#     min_alpha=0.0007,                                # Learning rate will linearly drop to min_alpha as training progresses.
#     sg=1,                                            # Training algorithm: skip-gram if sg=1, otherwise CBOW.
#     hs=0,                                            # If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
#     negative=20,                                     # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used. 
#     min_count=5,                                    # The model ignores all words with total frequency lower than this.
#     window=5,                                        # The maximum distance between the current and predicted word within a sentence.
#     sample=1e-3,                                     # The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
#     workers=8, 
#     cbow_mean=1,                                     # If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
#     null_word=0,                                     # 
#     trim_rule=None,                                  # 
#     sorted_vocab=1,                                  # If 1, sort the vocabulary by descending frequency before assigning word indices.
#     batch_words=10000,                               # Target size (in words) for batches of examples passed to worker threads (and thus cython routines).(Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
    
#     seed=1364,                                       # Seed for the random number generator.
#     # only for FastText (compare to word2vec)
#     #word_ngrams=1,                                   # If 1, uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec. 
#     #min_n=2,                                         # Minimum length of char n-grams to be used for training word representations.
#     #max_n=15,                                        # Max length of char ngrams to be used for training word representations. Set max_n to be lesser than min_n to avoid char ngrams being used.
#     #bucket=2000000                                  # Character ngrams are hashed into a fixed number of buckets, in order to limit the memory usage of the model. This option specifies the number of buckets used by the model.
)

## Preprocess before creating/updating LM

In [12]:
"""
def preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):
    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]
    return txt
"""

'\ndef preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):\n    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]\n    return txt\n'

In [13]:
"""
db_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)
db_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)
"""

'\ndb_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)\ndb_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)\n'

## OCR

In [14]:
list_sentences = db_sentence["ocr_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 5095


In [15]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]
len(flattened_list_sentences)

42059

In [16]:
flattened_list_sentences[0]

['first',
 'church',
 'i',
 'service',
 '1',
 'presbyterian',
 'i',
 "'",
 'anniversary',
 '1',
 'the',
 '150th',
 'anniveisarjj',
 'the',
 'first',
 'presb',
 '>',
 'terian',
 'chad',
 'service',
 'm',
 'australia',
 'will',
 'k',
 'celebrated',
 'to',
 'morrow',
 'special',
 'services',
 'will',
 'be',
 'be',
 'at',
 'the',
 'ebenezer',
 'p«sbjt«j',
 'church',
 ',',
 'six',
 'miles',
 'from',
 'wmdso',
 'the',
 'ebenezer',
 'church',
 'is',
 '»',
 'oldest',
 'church',
 'in',
 'australia',
 '?',
 'which',
 'services',
 'are',
 'still',
 '«',
 'ia',
 'the',
 'sen',
 'ice',
 'v',
 '.',
 '.',
 '11',
 'start',
 'at',
 'lp',
 '£',
 'with',
 'sacred',
 'mus.c',
 'ttajjl',
 '*',
 'followed',
 'at',
 '130',
 'p',
 'm',
 '«*£',
 'dedication',
 'of',
 'commemorate',
 'g',
 'tue',
 'moderator',
 'general',
 'of',
 'j',
 '«',
 'presbyterian',
 'church',
 'of',
 'au',
 '»',
 'the',
 'right',
 'rev',
 '^',
 '¿',
 'bz',
 'will',
 'conduct',
 'a',
 'special',
 'atfhe',
 'pä',
 'ws«.llbe^r',
 'the',
 

In [17]:
embedding_model_ocr = copy.deepcopy(embedding_model_orig)

In [18]:
embedding_model_ocr.workers = 8
embedding_model_ocr.vocabulary.min_count = 5
embedding_model_ocr.alpha = min_alpha_yet_reached*10.

In [19]:
embedding_model_ocr.build_vocab(flattened_list_sentences, update=True)
embedding_model_ocr.train(flattened_list_sentences, 
                          total_examples=embedding_model_ocr.corpus_count,
                          epochs=w2v_args.epochs,  
                          compute_loss=w2v_args.compute_loss)

2019-11-20 18:26:26,901 : INFO : collecting all words and their counts
2019-11-20 18:26:26,903 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 18:26:27,015 : INFO : PROGRESS: at sentence #10000, processed 434822 words, keeping 90410 word types
2019-11-20 18:26:27,110 : INFO : PROGRESS: at sentence #20000, processed 886062 words, keeping 160063 word types
2019-11-20 18:26:27,235 : INFO : PROGRESS: at sentence #30000, processed 1366349 words, keeping 227866 word types
2019-11-20 18:26:27,328 : INFO : PROGRESS: at sentence #40000, processed 1789264 words, keeping 284758 word types
2019-11-20 18:26:27,351 : INFO : collected 298566 word types from a corpus of 1896341 raw words and 42059 sentences
2019-11-20 18:26:27,352 : INFO : Updating model with new vocabulary
2019-11-20 18:26:27,498 : INFO : New added 19776 unique words (6% of original 318342) and increased the count of 19776 pre-existing words (6% of original 318342)
2019-11-20 18:26:27,636 : INFO 

2019-11-20 18:27:07,287 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 18:27:07,307 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 18:27:07,361 : INFO : EPOCH 4 - PROGRESS: at 99.09% examples, 120453 words/s, in_qsize 2, out_qsize 1
2019-11-20 18:27:07,362 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 18:27:07,391 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 18:27:07,437 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 18:27:07,438 : INFO : EPOCH - 4 : training on 1896341 raw words (1177915 effective words) took 9.8s, 120801 effective words/s
2019-11-20 18:27:08,594 : INFO : EPOCH 5 - PROGRESS: at 8.29% examples, 93158 words/s, in_qsize 15, out_qsize 0
2019-11-20 18:27:09,605 : INFO : EPOCH 5 - PROGRESS: at 22.32% examples, 116975 words/s, in_qsize 15, out_qsize 0
2019-11-20 18:27:10,721 : INFO : EPOCH 5 - PROGRESS: at 35.23% exampl

(5888494, 9481705)

In [20]:
print("\n\n[INFO] Save the model")
embedding_model_ocr.save("./w2v_005_EM_ocr_qual_3_4.model")

2019-11-20 18:27:17,221 : INFO : saving Word2Vec object under ./w2v_005_EM_ocr_qual_3_4.model, separately None
2019-11-20 18:27:17,222 : INFO : storing np array 'vectors' to ./w2v_005_EM_ocr_qual_3_4.model.wv.vectors.npy




[INFO] Save the model


2019-11-20 18:27:18,512 : INFO : not storing attribute vectors_norm
2019-11-20 18:27:18,513 : INFO : storing np array 'syn1neg' to ./w2v_005_EM_ocr_qual_3_4.model.trainables.syn1neg.npy
2019-11-20 18:27:19,753 : INFO : not storing attribute cum_table
2019-11-20 18:27:20,718 : INFO : saved ./w2v_005_EM_ocr_qual_3_4.model


## Corrected

In [21]:
list_sentences = db_sentence["corrected_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 5095


In [22]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]

In [23]:
flattened_list_sentences[0]

['first',
 'church',
 'service',
 'presbyterian',
 'anniversary',
 'the',
 '150th',
 'anniversary',
 'of',
 'the',
 'first',
 'presbyterian',
 'church',
 'service',
 'in',
 'australia',
 'will',
 'be',
 'celebrated',
 'to',
 'morrow',
 '.']

In [24]:
embedding_model_corrected = copy.deepcopy(embedding_model_orig)

In [25]:
embedding_model_corrected.workers = 8
embedding_model_corrected.vocabulary.min_count = 5
embedding_model_corrected.alpha = min_alpha_yet_reached*10.

In [26]:
embedding_model_corrected.build_vocab(flattened_list_sentences, update=True)
embedding_model_corrected.train(flattened_list_sentences, 
                                total_examples=embedding_model_corrected.corpus_count,
                                epochs=w2v_args.epochs,  
                                compute_loss=w2v_args.compute_loss)

2019-11-20 18:27:26,978 : INFO : collecting all words and their counts
2019-11-20 18:27:26,979 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 18:27:27,023 : INFO : PROGRESS: at sentence #10000, processed 217550 words, keeping 21037 word types
2019-11-20 18:27:27,064 : INFO : PROGRESS: at sentence #20000, processed 455085 words, keeping 31368 word types
2019-11-20 18:27:27,106 : INFO : PROGRESS: at sentence #30000, processed 695671 words, keeping 40549 word types
2019-11-20 18:27:27,147 : INFO : PROGRESS: at sentence #40000, processed 923693 words, keeping 47531 word types
2019-11-20 18:27:27,183 : INFO : PROGRESS: at sentence #50000, processed 1136699 words, keeping 53209 word types
2019-11-20 18:27:27,220 : INFO : PROGRESS: at sentence #60000, processed 1361413 words, keeping 58750 word types
2019-11-20 18:27:27,258 : INFO : PROGRESS: at sentence #70000, processed 1582999 words, keeping 63795 word types
2019-11-20 18:27:27,296 : INFO : collected 

2019-11-20 18:28:05,572 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-11-20 18:28:05,630 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-11-20 18:28:05,715 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 18:28:05,747 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 18:28:05,831 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 18:28:05,832 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 18:28:05,836 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 18:28:05,836 : INFO : EPOCH - 4 : training on 1807730 raw words (1216159 effective words) took 9.0s, 134484 effective words/s
2019-11-20 18:28:06,890 : INFO : EPOCH 5 - PROGRESS: at 9.82% examples, 111998 words/s, in_qsize 15, out_qsize 0
2019-11-20 18:28:07,932 : INFO : EPOCH 5 - PROGRESS: at 23.02% examples, 132840 words/s, in_qsize 16, out_qsize 0

(6077806, 9038650)

In [27]:
print("\n\n[INFO] Save the model")
embedding_model_corrected.save("./w2v_005_EM_corr_qual_3_4.model")

2019-11-20 18:28:14,124 : INFO : saving Word2Vec object under ./w2v_005_EM_corr_qual_3_4.model, separately None
2019-11-20 18:28:14,125 : INFO : storing np array 'vectors' to ./w2v_005_EM_corr_qual_3_4.model.wv.vectors.npy




[INFO] Save the model


2019-11-20 18:28:15,393 : INFO : not storing attribute vectors_norm
2019-11-20 18:28:15,393 : INFO : storing np array 'syn1neg' to ./w2v_005_EM_corr_qual_3_4.model.trainables.syn1neg.npy
2019-11-20 18:28:16,587 : INFO : not storing attribute cum_table
2019-11-20 18:28:17,600 : INFO : saved ./w2v_005_EM_corr_qual_3_4.model


# =================================

# Quality bands 1, 2

In [28]:
def cleanup(myrow, col_name):
    all_clean_rows = []
    for sent in myrow[col_name]:
        one_clean_row = []
        for token in sent:
            one_clean_row.append(token.lower())
        all_clean_rows.append(one_clean_row)
    return all_clean_rows

In [29]:
db_sentence = db_sentence_orig[(db_sentence_orig['quality_band'] == 1) | 
                               (db_sentence_orig['quality_band'] == 2)]
db_sentence['quality_band'].value_counts()

2    13953
1    11461
Name: quality_band, dtype: int64

In [30]:
db_sentence["ocr_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["ocr_sentencizer"], axis=1)
db_sentence["corrected_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["corrected_sentencizer"], axis=1)

/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
print(db_sentence.shape)
db_sentence.head()

(25414, 25)


,filePath,articleId,articleType,year,ocrText,humanText,corrected,str_similarity,str_length_humanText,str_length_ocrText,...,ocr_dict_lookup,ocr_dict_lookup_list,ocr_dict_perc,corrected_dict_lookup_list,corr_dict_perc,corrected_sentencizer_list,ocr_sentencizer_list,jaccard_similarity,ocr_sentencizer_cleaned,corrected_sentencizer_cleaned
1,./trove_overproof/datasets/dataset1/rawTextAnd...,18378453,Article ILLUSTRATED,1953,"FROM RIVER CROSSING TO END OF TRIÄÜ I ^PI A^H""...",FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,0.847561,746,820,...,"[[4, 5, 8, 2, 3, 2, -5, 1, 1, 2, -3, 1, -5, -6...","[4, 5, 8, 2, 3, 2, -5, 1, 1, 2, -3, 1, -5, -6,...",79.439252,"[4, 5, 8, 2, 3, 2, 5, 6, 1, -5, -6, 8, 4, 4, 5...",92.079208,"[FROM, RIVER, CROSSING, TO, END, OF, TRIAL, SP...","[FROM, RIVER, CROSSING, TO, END, OF, TRIÄÜ, I,...",0.305882,"[[from, river, crossing, to, end, of, triäü, i...","[[from, river, crossing, to, end, of, trial, s..."
2,./trove_overproof/datasets/dataset1/rawTextAnd...,18363627,Article,1953,"Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...",0.964119,641,630,...,"[[7, 10, -7, 6, 4, 4, 3, 3, 4, 5, 6, 4, 6, 5, ...","[7, 10, -7, 6, 4, 4, 3, 3, 4, 5, 6, 4, 6, 5, 2...",96.590909,"[7, 10, -7, 6, 4, 4, 3, 3, 4, 5, 6, 4, 6, 5, 2...",98.876404,"[Natural, Childbirth, Sir,-We, nurses, have, s...","[Natural, Childbirth, Sir,-We, nurses, have, s...",0.523810,"[[natural, childbirth, sir,-we, nurses, have, ...","[[natural, childbirth, sir,-we, nurses, have, ..."
5,./trove_overproof/datasets/dataset1/rawTextAnd...,18368961,Article,1953,"DIVORCE Before The Judge In Divorce, Mr Justic...","DIVORCE Before The Judge In Divorce, Mr. Justi...","DIVORCE Before The Judge In Divorce, Mr. Justi...",0.894176,1219,1121,...,"[[7, 6, 3, 5, 2, 7, 1, 2, 7, -5, 7, 4, 1, 1, -...","[7, 6, 3, 5, 2, 7, 1, 2, 7, -5, 7, 4, 1, 1, -6...",82.882883,"[7, 6, 3, 5, 2, 7, 1, 2, 1, 7, -5, 7, 4, 1, 1,...",86.486486,"[DIVORCE, Before, The, Judge, In, Divorce, ,, ...","[DIVORCE, Before, The, Judge, In, Divorce, ,, ...",0.147513,"[[divorce, before, the, judge, in, divorce, ,,...","[[divorce, before, the, judge, in, divorce, ,,..."
6,./trove_overproof/datasets/dataset1/rawTextAnd...,18355541,Article,1953,"BRITISH AIRLINER I MISSING ¡HALIFAX, Feb. 2 M....","BRITISH AIRLINER MISSING HALIFAX, Feb. 2 (A.A....","BRITISH AIRLINER MISSING HALIFAX, Feb. 2 (A.A....",0.829670,1274,1139,...,"[[7, 8, 1, 7, 1, 7, 1, 3, 1, 1, 4, 1, -3, 7, 3...","[7, 8, 1, 7, 1, 7, 1, 3, 1, 1, 4, 1, -3, 7, 3,...",88.194444,"[7, 8, 7, 7, 1, 3, 1, 1, 1, -6, 1, -3, 7, 3, 1...",95.731707,"[BRITISH, AIRLINER, MISSING, HALIFAX, ,, Feb, ...","[BRITISH, AIRLINER, I, MISSING, ¡, HALIFAX, ,,...",0.240449,"[[british, airliner, i, missing, ¡, halifax, ,...","[[british, airliner, missing, halifax, ,, feb,..."
7,./trove_overproof/datasets/dataset1/rawTextAnd...,18381450,Article,1953,I SCHOOL CHESS * Homebush Increased Ils lead o...,SCHOOL CHESS Homebush increased its lead over...,SCHOOL CHESS Homebush increased its lead over...,0.918264,991,955,...,"[[1, 6, 5, 1, 8, 9, 3, 4, 4, 8, 2, 3, 5, 5, 2,...","[1, 6, 5, 1, 8, 9, 3, 4, 4, 8, 2, 3, 5, 5, 2, ...",92.307692,"[6, 5, 8, 9, 3, 4, 4, 8, 2, 3, 5, 5, 2, 3, 1, ...",96.598639,"[SCHOOL, CHESS, Homebush, increased, its, lead...","[I, SCHOOL, CHESS, *, Homebush, Increased, Ils...",0.277612,"[[i, school, chess, *, homebush, increased, il...","[[school, chess, homebush, increased, its, lea..."


## Update a pre-trained LM

In [32]:
# args for Word2Vec
w2v_args = Namespace(
    epochs=5, 
    # only for Word2Vec
    compute_loss=True,                               # If True, computes and stores loss value which can be retrieved using get_latest_training_loss().

#     size=100,                                        # Dimensionality of the word vectors.
#     alpha=0.03,                                      # The initial learning rate.
#     min_alpha=0.0007,                                # Learning rate will linearly drop to min_alpha as training progresses.
#     sg=1,                                            # Training algorithm: skip-gram if sg=1, otherwise CBOW.
#     hs=0,                                            # If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
#     negative=20,                                     # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used. 
#     min_count=5,                                    # The model ignores all words with total frequency lower than this.
#     window=5,                                        # The maximum distance between the current and predicted word within a sentence.
#     sample=1e-3,                                     # The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
#     workers=8, 
#     cbow_mean=1,                                     # If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
#     null_word=0,                                     # 
#     trim_rule=None,                                  # 
#     sorted_vocab=1,                                  # If 1, sort the vocabulary by descending frequency before assigning word indices.
#     batch_words=10000,                               # Target size (in words) for batches of examples passed to worker threads (and thus cython routines).(Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
    
#     seed=1364,                                       # Seed for the random number generator.
#     # only for FastText (compare to word2vec)
#     #word_ngrams=1,                                   # If 1, uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec. 
#     #min_n=2,                                         # Minimum length of char n-grams to be used for training word representations.
#     #max_n=15,                                        # Max length of char ngrams to be used for training word representations. Set max_n to be lesser than min_n to avoid char ngrams being used.
#     #bucket=2000000                                  # Character ngrams are hashed into a fixed number of buckets, in order to limit the memory usage of the model. This option specifies the number of buckets used by the model.
)

## Preprocess before creating/updating LM

In [ ]:
"""
def preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):
    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]
    return txt
"""

In [ ]:
"""
db_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)
db_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)
"""

## OCR

In [54]:
import time

for i_model in range(40, 50):
    t1 = time.time()

    embedding_model_ocr = copy.deepcopy(embedding_model_orig)
    embedding_model_ocr.workers = 8
    embedding_model_ocr.vocabulary.min_count = 5
    embedding_model_ocr.alpha = min_alpha_yet_reached*10.
    
    db_sentence_sampled = db_sentence.sample(n=5095, replace=True)
    list_sentences = db_sentence_sampled["ocr_sentencizer_cleaned"].to_list()
    print('#sentences: {}'.format(len(list_sentences)))
    flattened_list_sentences = [val for sublist in list_sentences for val in sublist]
    print(i_model, len(flattened_list_sentences))
    embedding_model_ocr.build_vocab(flattened_list_sentences, update=True)
    embedding_model_ocr.train(flattened_list_sentences, 
                              total_examples=embedding_model_ocr.corpus_count,
                              epochs=w2v_args.epochs,  
                              compute_loss=w2v_args.compute_loss)
    print("\n\n[INFO] Save the model")
    embedding_model_ocr.save("./LMs/w2v_005_EM_ocr_qual_1_2_%05i.model" % i_model)
    print("total: {}".format(time.time() - t1))
    print("=================")

2019-11-20 21:16:35,247 : INFO : collecting all words and their counts
2019-11-20 21:16:35,248 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 21:16:35,342 : INFO : PROGRESS: at sentence #10000, processed 390636 words, keeping 56739 word types
2019-11-20 21:16:35,427 : INFO : PROGRESS: at sentence #20000, processed 819811 words, keeping 97237 word types


#sentences: 5095
40 51229


2019-11-20 21:16:35,509 : INFO : PROGRESS: at sentence #30000, processed 1211406 words, keeping 128411 word types
2019-11-20 21:16:35,598 : INFO : PROGRESS: at sentence #40000, processed 1593420 words, keeping 156189 word types
2019-11-20 21:16:35,695 : INFO : PROGRESS: at sentence #50000, processed 1991585 words, keeping 181801 word types
2019-11-20 21:16:35,707 : INFO : collected 185097 word types from a corpus of 2042938 raw words and 51229 sentences
2019-11-20 21:16:35,707 : INFO : Updating model with new vocabulary
2019-11-20 21:16:35,834 : INFO : New added 21544 unique words (10% of original 206641) and increased the count of 21544 pre-existing words (10% of original 206641)
2019-11-20 21:16:35,978 : INFO : deleting the raw counts dictionary of 185097 items
2019-11-20 21:16:35,982 : INFO : sample=0.001 downsamples 84 most-common words
2019-11-20 21:16:35,983 : INFO : downsampling leaves estimated 2619874 word corpus (143.8% of prior 1822450)
2019-11-20 21:16:37,137 : INFO : estim

2019-11-20 21:17:18,650 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 21:17:18,656 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 21:17:18,716 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 21:17:18,720 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 21:17:18,770 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 21:17:18,771 : INFO : EPOCH - 4 : training on 2042938 raw words (1376214 effective words) took 10.1s, 136901 effective words/s
2019-11-20 21:17:19,866 : INFO : EPOCH 5 - PROGRESS: at 8.87% examples, 103704 words/s, in_qsize 15, out_qsize 0
2019-11-20 21:17:21,002 : INFO : EPOCH 5 - PROGRESS: at 20.13% examples, 123304 words/s, in_qsize 15, out_qsize 0
2019-11-20 21:17:22,130 : INFO : EPOCH 5 - PROGRESS: at 31.09% examples, 129795 words/s, in_qsize 15, out_qsize 0
2019-11-20 21:17:23,268 : INFO : EPOCH 5 - PROGRESS: at 41.87% exa



[INFO] Save the model


...

2019-11-20 21:26:17,918 : INFO : PROGRESS: at sentence #30000, processed 1254470 words, keeping 131987 word types
2019-11-20 21:26:17,993 : INFO : PROGRESS: at sentence #40000, processed 1668768 words, keeping 162194 word types
2019-11-20 21:26:18,083 : INFO : PROGRESS: at sentence #50000, processed 2093993 words, keeping 189765 word types
2019-11-20 21:26:18,085 : INFO : collected 189928 word types from a corpus of 2097682 raw words and 50133 sentences
2019-11-20 21:26:18,086 : INFO : Updating model with new vocabulary
2019-11-20 21:26:18,216 : INFO : New added 21884 unique words (10% of original 211812) and increased the count of 21884 pre-existing words (10% of original 211812)
2019-11-20 21:26:18,366 : INFO : deleting the raw counts dictionary of 189928 items
2019-11-20 21:26:18,370 : INFO : sample=0.001 downsamples 84 most-common words
2019-11-20 21:26:18,371 : INFO : downsampling leaves estimated 2697390 word corpus (144.1% of prior 1872229)
2019-11-20 21:26:19,547 : INFO : estim

2019-11-20 21:27:02,950 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-11-20 21:27:02,992 : INFO : EPOCH 4 - PROGRESS: at 98.04% examples, 137486 words/s, in_qsize 4, out_qsize 1
2019-11-20 21:27:02,993 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 21:27:03,139 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 21:27:03,179 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 21:27:03,212 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 21:27:03,219 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 21:27:03,220 : INFO : EPOCH - 4 : training on 2097682 raw words (1416645 effective words) took 10.3s, 137012 effective words/s
2019-11-20 21:27:04,463 : INFO : EPOCH 5 - PROGRESS: at 7.83% examples, 91481 words/s, in_qsize 16, out_qsize 0
2019-11-20 21:27:05,469 : INFO : EPOCH 5 - PROGRESS: at 18.54% examples, 115793 words/s, in



[INFO] Save the model


2019-11-20 21:27:15,350 : INFO : not storing attribute vectors_norm
2019-11-20 21:27:15,351 : INFO : storing np array 'syn1neg' to ./LMs/w2v_005_EM_ocr_qual_1_2_00049.model.trainables.syn1neg.npy
2019-11-20 21:27:16,493 : INFO : not storing attribute cum_table
2019-11-20 21:27:17,529 : INFO : saved ./LMs/w2v_005_EM_ocr_qual_1_2_00049.model


total: 68.17731499671936


## Corrected

In [55]:
import time

for i_model in range(40, 50):
    t1 = time.time()
    
    embedding_model_corrected = copy.deepcopy(embedding_model_orig)
    embedding_model_corrected.workers = 8
    embedding_model_corrected.vocabulary.min_count = 5
    embedding_model_corrected.alpha = min_alpha_yet_reached*10.
        
    db_sentence_sampled = db_sentence.sample(n=5095, replace=True)
    list_sentences = db_sentence_sampled["corrected_sentencizer_cleaned"].to_list()
    print('#sentences: {}'.format(len(list_sentences)))
    flattened_list_sentences = [val for sublist in list_sentences for val in sublist]
    print(i_model, len(flattened_list_sentences))
    embedding_model_corrected.build_vocab(flattened_list_sentences, update=True)
    embedding_model_corrected.train(flattened_list_sentences, 
                                    total_examples=embedding_model_corrected.corpus_count,
                                    epochs=w2v_args.epochs,  
                                    compute_loss=w2v_args.compute_loss)
    
    print("\n\n[INFO] Save the model")
    embedding_model_corrected.save("./LMs/w2v_005_EM_corr_qual_1_2_%05i.model" % i_model)
    print("total: {}".format(time.time() - t1))
    print("=================")

2019-11-20 21:27:23,104 : INFO : collecting all words and their counts
2019-11-20 21:27:23,105 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 21:27:23,153 : INFO : PROGRESS: at sentence #10000, processed 246740 words, keeping 19704 word types
2019-11-20 21:27:23,196 : INFO : PROGRESS: at sentence #20000, processed 491912 words, keeping 29488 word types
2019-11-20 21:27:23,234 : INFO : PROGRESS: at sentence #30000, processed 735756 words, keeping 36394 word types
2019-11-20 21:27:23,272 : INFO : PROGRESS: at sentence #40000, processed 978693 words, keeping 42296 word types
2019-11-20 21:27:23,313 : INFO : PROGRESS: at sentence #50000, processed 1225546 words, keeping 47966 word types


#sentences: 5095
40 85298


2019-11-20 21:27:23,353 : INFO : PROGRESS: at sentence #60000, processed 1475520 words, keeping 52550 word types
2019-11-20 21:27:23,393 : INFO : PROGRESS: at sentence #70000, processed 1726077 words, keeping 56843 word types
2019-11-20 21:27:23,433 : INFO : PROGRESS: at sentence #80000, processed 1978915 words, keeping 60495 word types
2019-11-20 21:27:23,456 : INFO : collected 62326 word types from a corpus of 2110335 raw words and 85298 sentences
2019-11-20 21:27:23,457 : INFO : Updating model with new vocabulary
2019-11-20 21:27:23,530 : INFO : New added 18041 unique words (22% of original 80367) and increased the count of 18041 pre-existing words (22% of original 80367)
2019-11-20 21:27:23,655 : INFO : deleting the raw counts dictionary of 62326 items
2019-11-20 21:27:23,657 : INFO : sample=0.001 downsamples 68 most-common words
2019-11-20 21:27:23,658 : INFO : downsampling leaves estimated 2767005 word corpus (135.7% of prior 2038742)
2019-11-20 21:27:24,821 : INFO : estimated re

2019-11-20 21:28:03,915 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 21:28:03,960 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 21:28:04,012 : INFO : EPOCH 4 - PROGRESS: at 99.03% examples, 148194 words/s, in_qsize 2, out_qsize 1
2019-11-20 21:28:04,013 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 21:28:04,068 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 21:28:04,073 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 21:28:04,074 : INFO : EPOCH - 4 : training on 2110335 raw words (1430382 effective words) took 9.6s, 148650 effective words/s
2019-11-20 21:28:05,105 : INFO : EPOCH 5 - PROGRESS: at 8.11% examples, 113078 words/s, in_qsize 16, out_qsize 0
2019-11-20 21:28:06,166 : INFO : EPOCH 5 - PROGRESS: at 19.42% examples, 133849 words/s, in_qsize 15, out_qsize 0
2019-11-20 21:28:07,236 : INFO : EPOCH 5 - PROGRESS: at 30.97% examp



[INFO] Save the model


...

2019-11-20 21:36:25,226 : INFO : PROGRESS: at sentence #60000, processed 1466132 words, keeping 52097 word types
2019-11-20 21:36:25,264 : INFO : PROGRESS: at sentence #70000, processed 1707034 words, keeping 55857 word types
2019-11-20 21:36:25,301 : INFO : PROGRESS: at sentence #80000, processed 1947547 words, keeping 59729 word types
2019-11-20 21:36:25,324 : INFO : collected 61802 word types from a corpus of 2085441 raw words and 85568 sentences
2019-11-20 21:36:25,325 : INFO : Updating model with new vocabulary
2019-11-20 21:36:25,396 : INFO : New added 17857 unique words (22% of original 79659) and increased the count of 17857 pre-existing words (22% of original 79659)
2019-11-20 21:36:25,511 : INFO : deleting the raw counts dictionary of 61802 items
2019-11-20 21:36:25,513 : INFO : sample=0.001 downsamples 70 most-common words
2019-11-20 21:36:25,514 : INFO : downsampling leaves estimated 2735732 word corpus (135.8% of prior 2014870)
2019-11-20 21:36:26,678 : INFO : estimated re

2019-11-20 21:37:05,985 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 21:37:05,986 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 21:37:06,138 : INFO : EPOCH 4 - PROGRESS: at 99.49% examples, 142813 words/s, in_qsize 1, out_qsize 1
2019-11-20 21:37:06,139 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 21:37:06,202 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 21:37:06,203 : INFO : EPOCH - 4 : training on 2085441 raw words (1415785 effective words) took 9.9s, 142567 effective words/s
2019-11-20 21:37:07,401 : INFO : EPOCH 5 - PROGRESS: at 8.22% examples, 96598 words/s, in_qsize 15, out_qsize 0
2019-11-20 21:37:08,463 : INFO : EPOCH 5 - PROGRESS: at 19.66% examples, 123156 words/s, in_qsize 15, out_qsize 0
2019-11-20 21:37:09,514 : INFO : EPOCH 5 - PROGRESS: at 30.75% examples, 133163 words/s, in_qsize 15, out_qsize 0
2019-11-20 21:37:10,631 : INFO : EPOCH 5 - PRO



[INFO] Save the model


2019-11-20 21:37:17,173 : INFO : not storing attribute vectors_norm
2019-11-20 21:37:17,173 : INFO : storing np array 'syn1neg' to ./LMs/w2v_005_EM_corr_qual_1_2_00049.model.trainables.syn1neg.npy
2019-11-20 21:37:18,313 : INFO : not storing attribute cum_table
2019-11-20 21:37:19,350 : INFO : saved ./LMs/w2v_005_EM_corr_qual_1_2_00049.model


total: 60.251914978027344
